# Model Mismatch Testing

In [19]:
kchange         = 1
kkchange        = 1 #1.3;
Urchange        = 1 #0.7;
Mchange         = 1
deltaHchange    = 1


Bt      = 250      #(min)      %Batch time
dt      = 0.1                  #%Step size of process
n       = Bt/dt
t    = [0]                    #%Start time
sampt   = 2        #(min)      %Sampling time
j       = 2
k       = 1

# Process Parameters

In [20]:
k11     = 20.9057
k12     = 1e4
k21     = 38.9057
k22     = 17000
deltaH1 = -41840   #%(kJ/kmol)
deltaH2 = -25105   
MWA     = 30       #%(kg/kmol)
MWB     = 100
MWC     = 130
MWD     = 160
pr      = 1e3      #%(kg/m3)
r       = 0.5      #%(m)
Ur      = Urchange*40.842     #%(kJ/min m2 oC)
CpA     = 75.31               #%(kJ/kmol oC)
CpB     = 167.36
CpC     = 217.57
CpD     = 334.73
Fj      = 0.348    #%(m3/min)
pj      = 1e3      #%(kg/m3)
Cpj     = 1.8828   #%kJ/(kg oC)
Vj      = 0.6912   #%m3

# Process Initial Conditions

In [21]:
MA   = [Mchange*12];
MB   = [12];
MC   = [0];
MD   = [0];
Tr   = [20];
Tj   = [20];
Tj   = [20];
Tjsp = [95];
Qr   = [0];
Qj   = [0];

# Initial Of Measurement Output #

# Parameters using in GMC
Note: These parameter values are from A. Arpornwichanop et al., On-line
dynamic optimization and control strategy for improving the performance of
batch reactors

In [22]:
MR      = 1560     #%(kg)
CPR     = 1.8828   #%(kJ/kg oC)
UA      = 254.8541 #%(kJ/min oC)

Parameter for GMC controller

In [23]:
Trsp= [95];
Tju = [20];
ej  = [0];

Tuning parameter

In [24]:
Tauj   = 0.2;
K1     = 0.72;
K2     = 10^-4;

# Start: Process Dynamics

In [29]:
for i in range(0,n):
   
    '''Process'''
    k1      = kkchange*exp(k11-(k12/(Tr[i]+273.15)));
    k2      = exp(k21-(k22/(Tr[i]+273.15)));
    R1      = k1*MA[i]*MB[i];
    R2      = k2*MA[i]*MC[i];
    MA[i+1] = MA[i]+dt*(-R1-R2);        #%---discrete form---%   %dMA/dt = -R1-R2
    MB[i+1] = MB[i]+dt*(-R1);
    MC[i+1] = MC[i]+dt*(R1-R2);
    MD[i+1] = MD[i]+dt*(R2);
    Qr[i+1] = -deltaH1*R1-deltaH2*R2;   #%exothermic
    Wr      = MWA*MA[i]+MWB*MB[i]+MWC*MC[i]+MWD*MD[i];
    Vr      = Wr/pr;
    Ar      = 2*Vr/r;
    Qj[i+1] = Ur*Ar*(Tj[i]-Tr[i]);      
    Mr      = MA[i]+MB[i]+MC[i]+MD[i];
    Cpr     = (CpA*MA[i]+CpB*MB[i]+CpC*MC[i]+CpD*MD[i])/Mr;
    Tr[i+1] = Tr[i]+dt*((Qr[i]+Qj[i+1])/Mr/Cpr);
    Tj[i+1] = Tj[i]+dt*((Fj*pj*Cpj*(Tjsp[i]-Tj[i])-Qj[i+1])/Vj/pj/Cpj);
    
    Trsp[i+1] = Trsp[i];
    Tjsp[i+1] = Tjsp[end];

    if mod(i,sampt) == 0:
        
        ej[j+1]  = Trsp[i+1]-Tr[i+1];
        sum[j+1] = sum[j]+ej[j+1];
        
        x1  = (Wr*CPR/(Ur*Ar));
        x2  = (K1*(ej[j+1]));
        x3  = (K2*(sum[j+1])*dt);
        x4  = (Qr[end]/(Ur*Ar));
        
        Tju[i+1] = Tr[i+1]+(x1*(x2+x3))-x4;
        
        a   = Tju[i];
        b   = Tju[i+1];
        
        Tjsp[i+1] = a+(Tauj*((b-a)/dt));
        
        if Tjsp[i+1] <= 0:      #%lower boundary
            Tjsp[i+1] = 0;
                
                 
                        
        elif Tjsp[i+1] >= 120:   #%upper boundary
            Tjsp[i+1] = 120;    
        end
            
        j = j+1;
    else:
        
        Tj[i+1]= Tj[end];
    break
t[i+1] = t[i]+dt;
    


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
figure(1);
plot(t,MA,'r.-',t,MB,'b.-',t,MC,'k.-',t,MD,'g.-');
title('Concentration profile (GMC)');
xlabel ('Time (min)'); ylabel ('Concentration (kmol)');
legend('Ma','Mb','Mc','Md');

figure(2);
plot(t,Qr,'r.-');
title('Heat-release profile (GMC)');
xlabel ('Time (min)'); ylabel ('Heat transfer (kJ/min)');
legend('Qr');

figure(3);
plot(t,Tr,'r.-',t,Trsp,'b.-',t,Tj,'k.-',t,Tjsp,'g.-');
title('Temperature profile (GMC)');
xlabel ('Time (min)'); ylabel ('Temperature (oC)');
axis([t(1) Bt 0 130])
legend('Tr','Trsp','Tj','Tjsp');